Author : Jeonghun Yoon

이번 노트북의 주제는 **Autoencoder** 입니다. **Autoencoder**는 input data와 output data가 비슷한(거의 동일한) 신경망 구조입니다. 이 신경망 구조의 핵심 아이디어는 input data에 내제되어 있는 특성 즉 *latent representation*을 잘 찾아내는 것입니다. 다양한 autoencoder 응용 모델 중에서, machine translation에 사용되는, Sequence-to-Sequence 모델인 **RNN Encoder-Decoder**에 대해서 공부하도록 하겠습니다. 

Reference는 아래의 논문들입니다.
 1. [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
 2. [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)
 
Sequence-to-Sequnece 모델인 **RNN Encoder-Decoder**의 input을 $(x_1,...,x_{T})$, output을 $(y_1,...,y_{T'})$ 라고 하겠습니다. 목표는 $p(y_1,...,y_{T'}|x_1,...,x_T)$ 인 조건부 확률모델을 학습하는 것입니다 [1],[2].
RNN Encoder-Decoder 모델은 2개의 phase로 구성됩니다.
 - Encoder : input sequence $(x_1,...,x_{T})$를 순서대로 읽는 RNN 모델입니다. RNN 모델이기 때문에 hidden state를 가지고 있고, hidden state의 update 조건은 $\textbf{h}_t=f(\textbf{h}_{t-1}, x_t)$ 입니다. $f$는 non-linear activation 함수 입니다.
 - Decoder : (output sequence에서) 다음에 나올 symbol인 $y_t$를 예측(prediction)하면서 output seqeunce를 생성하는 학습모델입니다. 기본적인 RNN과 달리 이전 time stamp에서의 출력이 RNN의 입력으로 사용됩니다. hidden state의 update 조건은 $\textbf{h}_t=f(\textbf{h}_{t-1}, y_{t-1}, \textbf{c})$ 이며 $\textbf{c}$에는 input sequence 전체의 정보(fixed dimensional representation)가 축약되어 있습니다. 따라서 output sequence에서, 다음 symbol에 대한 조건부 확률은 $P(y_t|y_{t-1},...,y_1,\textbf{c})=g(\textbf{h}_t, y_{t-1}, \textbf{c})$ 입니다. $g$는 유효한 확률값을 생성해야 하므로 softmax function과 같은 함수가 될 것입니다.
 
따라서 우리가 구하고자 하는 조건부 확률모델은 아래와 같습니다.
$$p(y_1,...,y_{T'}|x_1,...,x_T)=\prod_{t=1}^{T'}p(y_t|y_{t-1},...,y_1,\textbf{c})$$

# 1. Tensorflow version

정수 계산기를 RNN Encoder-Decoder를 이용하여 구현해보도록 하겠습니다.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import typing

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

##### const.py

In [ ]:
BATCH_SIZE = 128
NUM_EPOCHS = 300
INPUT_SEQUENCE_LENGTH = 7
EMBEDDING_DIM = 12
OUTPUT_SEQUENCE_LENGTH = 4

##### data_helper.py

In [ ]:
"""
1. Data preprocessing
=====================

Tutorial에서 사용할 데이터 sample 조건은 다음과 같습니다. 
 - 덧셈 연산
 - 정수의 최대 자리수(digits)는 3자리

ex) 
123+456=579, 87+138=225, ... 
 - '123+456', '87+138'은 input sequence
 - '579', '225'는 output sequence
"""  

def _get_num(digits):
    num = ''
    for _ in range(np.random.randint(1, digits+1)):
        num += np.random.choice(list('0123456789'))
    return int(num)

def _padding(seq, max_len):
    """Fix the input sequence length for padding the input sequence."""
    return seq + ' ' *(max_len - len(seq))

def _get_input_n_output_seq(digits):
    """Input sequence와 output sequence pair를 생성합니다."""
    num1 = _get_num(digits)
    num2 = _get_num(digits)
    question = '{}+{}'.format(num1, num2)
    answer = str(num1 + num2)
    return list(_padding(question, 2 * digits + 1)), list(_padding(answer, digits + 1))

def create_num_seqs(digits=3, data_size=50000):
    input_seqs = []
    output_seqs = []

    for _ in list(range(data_size)):
        input_seq, output_seq = _get_input_n_output_seq(digits)
        input_seqs.append(input_seq)
        output_seqs.append(output_seq)

    return input_seqs, output_seqs

def create_onehot_seqs(input_num_seqs, output_num_seqs):
    """Create one hot vectors"""
    label_bin = LabelBinarizer()
    label_bin.fit(list(' +0123456789'))

    input_onehot_seqs = [label_bin.transform(seq) for seq in input_num_seqs]
    output_onehot_seqs = [label_bin.transform(seq) for seq in output_num_seqs]
    
    return input_onehot_seqs, output_onehot_seqs

In [ ]:
# Voca table
voca_lookup = {}
for idx, ch in enumerate(list(' +0123456789')):
    voca_lookup[idx] = ch

In [ ]:
# Num sequences
input_num_seqs, output_num_seqs = create_num_seqs()

In [ ]:
# Onehot sequences
input_onehot_seqs, output_onehot_seqs = create_onehot_seqs(input_num_seqs, output_num_seqs)

In [ ]:
# LSTM(encoder) input shape
input_onehot_seqs = np.array(input_onehot_seqs, dtype=np.float32).reshape([-1, INPUT_SEQUENCE_LENGTH, EMBEDDING_DIM])

##### hparams.py

In [ ]:
num_hidden = 128
forget_bias = 1.0
lr = 0.001

##### train.py

In [ ]:
"""
2. Encoder and decoder
======================

Incoder와 decoder를 정의합니다.
Layer가 1인 encoder입니다.
"""

def inference(X, t, batch_size, is_training):
    # 1. Encoder
    encoder = tf.nn.rnn_cell.LSTMCell(num_hidden, forget_bias=forget_bias)
    initial_state = encoder.zero_state(batch_size, tf.float32)
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder, 
                                                        X, 
                                                        initial_state=initial_state, 
                                                        dtype=tf.float32)
    final_encoder_output = encoder_outputs[:, -1, :]

    # 2. Decoder : 학습의 경우와 학습이 아닌 경우를 나누어야 합니다.
    with tf.variable_scope('Decoder'):
        decoder = tf.nn.rnn_cell.LSTMCell(num_hidden, forget_bias=forget_bias)
        # Variables for logits
        V = tf.Variable(tf.truncated_normal([num_hidden, EMBEDDING_DIM], stddev=0.01, dtype=tf.float32))
        c = tf.Variable(tf.zeros([EMBEDDING_DIM], dtype=tf.float32))
        if is_training is True:
            """학습인 경우는, target sequence가 input으로 주어집니다. 따라서 input을 그대로 사용하면 됩니다."""
            decoder_outputs, decoder_states = tf.nn.dynamic_rnn(decoder,
                                                                t, 
                                                                initial_state=encoder_state,
                                                                dtype=tf.float32)
            # First element of output : Encoder final output
            # Previous output is the input of current state!
            outputs = tf.concat([tf.reshape(final_encoder_output, [-1, 1, num_hidden]), decoder_outputs[:,0:OUTPUT_SEQUENCE_LENGTH-1,:]], axis=1)
            # Logits
            logits = tf.einsum('ijk,kl->ijl', outputs, V) + c
            return logits
        else:
            """학습이 아닌경우는, 직전의 output을 target sequence를 예측하기 위한 input으로 다시 사용합니다."""
            state = encoder_state
            decoder_outputs = [final_encoder_output]
            outputs = []
            for i in range(1, OUTPUT_SEQUENCE_LENGTH):
                if i > 1:
                    tf.get_variable_scope().reuse_variables()
                logit_val = tf.matmul(decoder_outputs[-1], V) + c
                prob_val = tf.nn.softmax(logit_val)
                # 모델의 출력값
                outputs.append(prob_val)
                # 이전의 모델 출력값이, 입력으로 사용됩니다. 따라서, 입력값과 동일하게 onehot vector로 만들어줍니다.
                prob_one_hot = tf.one_hot(tf.argmax(prob_val, -1), depth=OUTPUT_SEQUENCE_LENGTH)
                # 현재의 모델 출력값
                output, state = decoder(prob_one_hot, state)
                decoder_outputs.append(output)
            # 모델의 마지막 출력값을 구한다.
            final_logit = tf.matmul(decoder_outputs[-1], V) + c
            final_prob = tf.nn.softmax(final_logit)
            outputs.append(final_prob)
            outputs = tf.reshape(tf.concat(outputs, axis=1), [-1, OUTPUT_SEQUENCE_LENGTH, EMBEDDING_DIM])
            return outputs

In [ ]:
"""
3. Learning functions
=====================

학습에 사용되는 function들을 정의합니다.
"""

def loss(logits, labels):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))

def train_op(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    return optimizer.minimize(loss)

def accuracy(y, t):
    y = tf.nn.softmax(y)
    correct_pred = tf.equal(tf.argmax(y, -1), tf.argmax(t, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    return accuracy

In [ ]:
# test_X = input_onehot_seqs[:5]
# test_y = output_onehot_seqs[:5]

In [ ]:
# sess.run(logits, feed_dict={X: test_X, y: test_y})

In [ ]:
"""
4. Placeholder 정의
=====================

학습에 사용되는 Placeholder들을 정의합니다.
"""
X = tf.placeholder(dtype=tf.float32, shape=(None, INPUT_SEQUENCE_LENGTH, EMBEDDING_DIM))
t = tf.placeholder(dtype=tf.float32, shape=(None, OUTPUT_SEQUENCE_LENGTH, EMBEDDING_DIM))
batch_size = tf.placeholder(dtype=tf.int32, shape=[])
is_training = tf.placeholder(dtype=tf.bool, shape=[])

In [ ]:
"""
5. Train step 정의
=====================

Inference, loss, optimizer를 정의합니다.
"""
y = inference(X, t, batch_size, is_training)
loss_val = loss(y, t)
train_step = train_op(loss_val)
acc_val = accuracy(y, t)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(input_onehot_seqs, output_onehot_seqs, test_size=0.2)

##### executor.py

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(NUM_EPOCHS):
    samp_X, samp_y = shuffle(train_X, train_y)
    num_batch = int(len(train_X) / BATCH_SIZE)
    for i in range(num_batch):
        start = i * BATCH_SIZE
        end = start + BATCH_SIZE
        sess.run(train_step, feed_dict={
            X: samp_X[start:end],
            t: samp_y[start:end],
            batch_size: BATCH_SIZE,
            is_training: True
        })
    val_loss = loss_val.eval(session=sess, feed_dict={
        X: test_X,
        t: test_y,
        batch_size: len(test_X),
        is_training: False
    })
    acc_loss = acc_val.eval(session=sess, feed_dict={
        X: test_X,
        t: test_y,
        batch_size: len(test_X),
        is_training: False
    })
    print('epoch: {}, loss: {}, accuracy: {}'.format(epoch, val_loss, acc_loss))
    # 검증 데이터
    if epoch > 150:
        for i in range(3):
            idx = np.random.randint(0, len(test_X))
            question = test_X[idx].reshape([-1, INPUT_SEQUENCE_LENGTH, EMBEDDING_DIM])
            answer = test_y[idx].reshape([-1, OUTPUT_SEQUENCE_LENGTH, EMBEDDING_DIM])
            prediction = y.eval(session=sess, feed_dict={
                X: question,
                t: answer,
                batch_size: 1,
                is_training: True
            })
            question = question.argmax(axis=-1)
            answer = answer.argmax(axis=-1)
            prediction = np.argmax(prediction, axis=-1)

            question = ''.join(voca_lookup[i] for i in question[0])
            answer = ''.join(voca_lookup[i] for i in answer[0])
            prediction = ''.join(voca_lookup[i] for i in prediction[0])

            print('==========')
            print('Q: {}, A: {}, P: {}'.format(question, answer, prediction))
            if answer == prediction: 
                print('True')
            else:
                print('False')